# PanGaia_Control: A Python Toolkit to prepare a Gaia DR2 Control Sample
**Goals**: 
To create a Control Sample from an user-provided initial sample.


**Methodology**:
1. Cross-matching (by 2MASS ID) the initial sample against the Gaia DR2 (via astroquery.gaia). 
2. Applying 3 quality selection criteria ($\varpi$ S/N, vissibility periods, RUWE)
3. Identifying outliers (if any) in $\varpi$, $\mu_{\alpha}^*$, and $\mu_{\delta}$ via a $\sigma$-threesold flexible criteria.

**User Inputs**: 
The initial sample: One (1) .vot table containing the 2MASS Source_IDs and J-band magnitudes of the sample.

**Products**: 
The Control sample.

**Notes**: 
1. The code automatically searches for initial tables located in the current directory as "sample_ini* 2MASS*"
2. This project is expands the work presented by Canovas et al. 2019, A&A (https://www.aanda.org/articles/aa/abs/2019/06/aa35321-19/aa35321-19.html).

In [1]:
from astropy.table            import Table, join, Column, MaskedColumn

# Python Basics ========================
import warnings, itertools, sys
sys.path.append('../libraries')

# Custom ===============================
from library_control      import sample_control as lib_control
from library_gaia         import sample_gaia    as lib_gaia
from library_plotters     import Basic_Plotters as lib_plot

# PyEsasKy =============================
from pyesasky.pyesasky import ESASkyWidget

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


# 1.- Run ESASky

In [2]:
# # Run ESASKY ===========================================
# esasky = ESASkyWidget()
# esasky

In [3]:
# back_dict = {'label':'AllWISE color', 'url': 'http://cdn.skies.esac.esa.int/AllWISEColor/'}
# back_dict = {'label':'AllWISE color', 'url': 'http://cdn.skies.esac.esa.int/AllWISEColor/'}
# # esasky.setHiPS(back_dict['label'], back_dict['url'])
# # esasky.setHiPS('Fermi color')

## 2.- Load Initial Sample

In [4]:
# Load initial sample for analysis =====================
control = lib_control()
control.read_sample_ini()
control.sample_ini_cat[0:3]

No Initial samples found in current directory.


AttributeError: 'sample_control' object has no attribute 'sample_ini_name'

## 3.- Cross-Match Initial Sample against Gaia DR2: ADQL query

In [ ]:
# Download Sample & Apply Quality cuts =================
control.run_gaia_cross_match()
control.threshold_vissibilities(visibility_min = 7)  # This code is also interactive (e.g. try control.threshold_vissibilities())
control.threshold_ruwe(ruwe_max = 1.40)
control.threshold_parallax_SN(par_SN_min = 10)
control.add_distance()
control.add_galactic()
control.add_3D_galactic()
control.plot_jband_hist()
control.cat[0:3]

## 4.- Generate Histograms

In [ ]:
# Generate histograms ==================================
figures    = lib_plot()
figures.load_gaia_cat(control.cat)
fname      = control.sample_ini_name + '_hist_astrometry.pdf'
histograms = figures.plot_3_hist(inp_col_1 = 'distance', fig_nm = fname, x1_bins=5, x2_bins=5, x3_bins=5, fontsize = 40, Gauss_fit = True)
# histograms = figures.plot_3_hist(fig_nm = fname, x1_bins=5, x2_bins=5, x3_bins=5, fontsize = 40, Gauss_fit = True)

## 5.- Create Control Sample

In [ ]:
# Remove Para, pmRA, pmDec Outliers ====================
control.remove_hist_outliers(histograms_dat=histograms, verbose = True)

# Generate new histograms ==============================
figures      = lib_plot()
figures.load_gaia_cat(control.cat)
fname        = control.sample_con_name + '_hist_astrometry.pdf'
histograms_2 = figures.plot_3_hist(inp_col_1 = 'distance', fig_nm = fname, x1_bins=3, x2_bins=5, x3_bins=5, fontsize = 40, Gauss_fit = True)
# histograms_2 = figures.plot_3_hist(fig_nm = fname, x1_bins=3, x2_bins=5, x3_bins=5, fontsize = 40, Gauss_fit = True)

# Save Control Sample ==================================
control.save_control_sample()

In [ ]:
# Examine Velocity & Histograms ========================
figures  = lib_plot()
figures.load_gaia_cat(control.cat)
figures.plot_2d_and_hist(col_hist = 'distance', fig_nm = control.sample_con_name + '_pmotions_and_pardist.pdf')
figures.plot_2d_and_hist(col_hist = 'parallax', fig_nm = control.sample_con_name + '_pmotions_and_parhist.pdf')

In [ ]:
# Prepare Cone Search ==================================
control.to_cone_search()
control.ADQL_pars_table

In [ ]:
# Show Control Table on Sky ============================
figures  = lib_plot()
figures.load_gaia_cat(control.cat)
figures.send_to_ESASky(esasky, catalogueName = 'Control', background='WISE', color='white', radius = control.ADQL_pars['radius'].value * 2)